In [1]:
import pandas as pd
import datetime
year=['2017','2018','2019','2020']#设置年份
# month=[('0101','0331'),('0401','0630'),('0701','0930'),('1001','1231')]#设置时间段
m=['-01','-02','-03','-04','-05','-06','-07','-08','-09','-10','-11','-12']#设置月份
def get__score(df_basic_filled_list,d,t):#对单一指标进行排序
    df=df_indicator_filled_list
    df=df_indicator_filled_list[t[0:4]][t[4:]]
    df=df.loc[:,['ts_code',d]]
    dg=df.groupby('ts_code')
    dg=dg.mean()
    dg=dg.sort_values(d,ascending=False)
    rank=dg.rank()
    return rank
def unit_socre(df,ind,t):#联合单一指标获得联合指标排序
    tmp=get__score(df,ind[0],t)
    rank=tmp
    for i in range(1,len(ind)):
        tmp=get__score(df,ind[i],t)
        rank = pd.merge(rank,tmp,how='inner',on='ts_code')
    rank['score']=rank.apply(lambda x:x.sum(),axis=1)
    return rank
# =============================================================================
#step1.读取数据 
# =============================================================================
#df_indicator=pd.read_csv('D:\\金融建模\\数据\\import_data\\fin_indicator.csv')#读入数据
#df_indicator=pd.read_csv('D:\\金融建模\\数据\\import_data\\table.csv')#读入数据
df_indicator=pd.read_csv('daily_basic.csv')#读入数据
#df_basic=df_basic.loc[:,['ann_date','total_share','ts_code']]
# =============================================================================
# step2.数据基本处理
# =============================================================================
df_indicator_filled=df_indicator.fillna(0)#填充空缺
df_indicator_filled['trade_date']=pd.to_datetime(df_indicator_filled['trade_date'],format='%Y%m%d')#设置时间戳
df_indicator_filled=df_indicator_filled.set_index('trade_date')#设置index为时间序列
df_indicator_filled_list={}#字典保存按年份和月份分割的Dataframe
# =============================================================================
# step3.划分数据生成字典
# =============================================================================
for i in year:
    month_dict={}
    for j in m:
        month_dict[j]=df_indicator_filled.loc[i+j,:]
    df_indicator_filled_list[i]=month_dict
# =============================================================================
# 设置指定指标
#fina_indicator=['roa','roe','debt_to_assets','bps']#选取的指标
#table=['total_share']
daily=['pe','pb','ps','total_mv']
# =============================================================================
score_indicator=unit_socre(df_indicator_filled_list,daily,'2017-04')#获得指定指标序列和指定时间的打分
select=score_indicator.sort_values(by='score',ascending=False).head(100)
print(select.index)
# t='2020-06'
# d='roa'
# df=df_indicator_filled_list
# df=df_indicator_filled_list[t[0:4]][t[4:]]
# df=df.loc[:,['ts_code',d]]
# dg=df.groupby('ts_code')
# dg=dg.mean()
# dg=dg.sort_values(d,ascending=False)
# rank=dg.rank()
# rank=pd.DataFrame(index=)
# for i in fina_indicator:
#     tmp=get__score(df,i,t)
#     rank = pd.merge(rank,tmp,how='inner',on='ts_code')

Index(['000503.SZ', '000820.SZ', '300122.SZ', '600233.SH', '002120.SZ',
       '002625.SZ', '600161.SH', '002522.SZ', '000025.SZ', '300136.SZ',
       '600435.SH', '600764.SH', '600876.SH', '600570.SH', '002468.SZ',
       '002346.SZ', '600698.SH', '600610.SH', '300601.SZ', '002797.SZ',
       '000813.SZ', '002127.SZ', '002095.SZ', '600679.SH', '000923.SZ',
       '000068.SZ', '000697.SZ', '300308.SZ', '300482.SZ', '002044.SZ',
       '002451.SZ', '002252.SZ', '002180.SZ', '002569.SZ', '002387.SZ',
       '300101.SZ', '603881.SH', '000831.SZ', '601020.SH', '300085.SZ',
       '300506.SZ', '000766.SZ', '600760.SH', '603986.SH', '002694.SZ',
       '601069.SH', '002622.SZ', '002515.SZ', '300191.SZ', '002530.SZ',
       '300142.SZ', '300558.SZ', '603058.SH', '002676.SZ', '300529.SZ',
       '600393.SH', '002575.SZ', '600771.SH', '300474.SZ', '600158.SH',
       '600779.SH', '603859.SH', '300496.SZ', '300618.SZ', '300302.SZ',
       '600436.SH', '000806.SZ', '300628.SZ', '600110.SH', '6037

In [2]:
l=list(select.index)

In [3]:
data=pd.read_csv("company_label.csv")

In [4]:
import re

In [5]:
data["code"]=data["sec_code"].apply(lambda x:re.findall(r"(.+)_.+_.+",x)[0]+"."+re.findall(r".+_(.+)_.+",x)[0])

In [6]:
data1=data[data["code"].isin(l)]

In [7]:
data1["time"]=pd.to_datetime(data1["news_time"])

<ipython-input-7-1842ee83424d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["time"]=pd.to_datetime(data1["news_time"])


In [8]:
t=pd.DataFrame(columns=["pos","neg","neu","code"])

In [9]:
t=pd.DataFrame(columns=["pos","neg","neu","公司","code"])
for i in data1["code"].unique():
    t1=data1[data1["code"]==i]
    t1.index=data1[data1["code"]==i]["time"]
    t2=t1.resample("1d")["pos","neg","neu"].mean()
    t2["code"]=i
    t2["公司"]=data1[data1["code"]==i].company_name.unique()[0]
    t=pd.concat([t,t2])

In [10]:
t.fillna(0,inplace=True)

In [11]:
df=pd.read_csv("event_label.csv")
event=df.merge(data1,on="news_id",how="right")

In [12]:
import numpy as np

In [13]:
for i in event["event_name"].unique()[1:]:
    event[i]=event["event_name"].apply(lambda x :1 if x==i else 0)
    event[i]=event[i].astype(np.uint8)

In [14]:
ls=list(event.columns[13:])
ls.append("公司")
ls.append("code")

In [15]:
len(event.code.unique())

100

In [16]:
T=pd.DataFrame(columns=ls)
for i in event["code"].unique():
    T1=event[event["code"]==i]
    T1.index=event[event["code"]==i]["time"]
    T2=T1.resample("1d")[list(event.columns[13:])].sum()
    T2["code"]=i
    T2["公司"]=event[event["code"]==i].company_name.unique()[0]
    T=pd.concat([T,T2])

In [22]:
print(t)
print(T)
t.to_csv("smallt.csv",index=False)
T.to_csv("bigT.csv",index=False)
# final=pd.concat([t,T.iloc[:,:-2]],axis=1)

                 pos       neg       neu   公司       code
2017-01-01  0.268822  0.183989  0.547200  新华网  603888.SH
2017-01-02  0.151290  0.061890  0.786810  新华网  603888.SH
2017-01-03  0.219722  0.098174  0.682117  新华网  603888.SH
2017-01-04  0.309385  0.117896  0.572712  新华网  603888.SH
2017-01-05  0.265543  0.085724  0.648752  新华网  603888.SH
...              ...       ...       ...  ...        ...
2020-06-19  0.188033  0.026467  0.785500  美利云  000815.SZ
2020-06-20  0.000000  0.000000  0.000000  美利云  000815.SZ
2020-06-21  0.000000  0.000000  0.000000  美利云  000815.SZ
2020-06-22  0.000000  0.000000  0.000000  美利云  000815.SZ
2020-06-23  0.036600  0.038700  0.924700  美利云  000815.SZ

[126445 rows x 5 columns]
           改革 遭监管 整改 商品价格稳定 大盘下跌 A+轮融资 筹划收购股权 收购股权 遭问询 补贴调整  ... 担保制度 增发保荐报告  \
2017-01-01  0   0  0      0    0     0      0    0   0    0  ...    0      0   
2017-01-02  0   0  0      0    0     0      0    0   0    0  ...    0      0   
2017-01-03  0   0  0      0    0     0      0    

In [21]:
final=pd.concat([t,T.iloc[:,:-2]],axis=1)
final.to_csv("final_2.csv",index=False)

MemoryError: Unable to allocate 90.1 GiB for an array with shape (963, 12558485) and data type object

In [ ]:
df1=pd.read_csv("industry.csv")

In [ ]:
df1["code"]=df1["company_code"].apply(lambda x:re.findall(r"(.+)_.+_.+",x)[0]+"."+re.findall(r".+_(.+)_.+",x)[0])